<a href="https://colab.research.google.com/github/hari-19/Smart_India_Hackathon-2020/blob/master/SIH_Deep_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [0]:
#Data Set needs to be Loaded

In [0]:
#Returns tanh of Data

def tanh(data):
  data = np.tanh(data)
  return data


In [0]:
#Returns ReLU of Data

def relu(data):
    data = np.where(data>0,data,0)
    return data

In [0]:
#Returns Sigmoid of Data

def sigmoid(data):
    data = 1 + np.exp(-data)
    data = 1/data
    return data

In [0]:
#Parameter initialization

    # Arguments:
    # layer_dims -- python array (list) containing the dimensions of each layer in the network
    
    # Returns:
    # parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
    #                 Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
    #                 bl -- bias vector of shape (layer_dims[l], 1)

def initialize_parameters(layer_dims):
    parameters={}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1)) 
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

    return parameters

In [0]:
    # The linear part of a layer's forward propagation.

    # Arguments:
    # A -- activations from previous layer : (size of previous layer, number of examples)
    # W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    # b -- bias vector, numpy array of shape (size of the current layer, 1)

    # Returns:
    # Z -- the input of the activation function
    # cache -- a python tuple containing "A", "W" and "b"

def linear_forward(A, W, b):
    Z = np.dot(W,A) + b

    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [0]:
    # The forward propagation for the LINEAR->ACTIVATION layer

    # Arguments:
    # A_prev -- activations from previous layer : (size of previous layer, number of examples)
    # W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    # b -- bias vector, numpy array of shape (size of the current layer, 1)
    # activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu" or "tanh"(will be coded later)

    # Returns:
    # A -- the output of the activation function
    # cache -- a python tuple containing "linear_cache" and "activation_cache";

def linear_activation_forward(A_prev, W, b, activation):
   
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A = relu(Z)
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, Z)

    return A, cache

In [0]:

    # Forward propagation for the [Linear --> Sigmoid] * (L-1) -> [LINEAR->ReLU] computation
    
    # Arguments:
    # X -- data, numpy array of shape (input size, number of examples)
    # parameters -- output of initialize_parameters_deep()
    
    # Returns:
    # AL -- last post-activation value
    # caches -- list of caches containing:
    #             every cache of linear_activation_forward() (there are L-1 of them, indexed from 0 to L-1)

def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters["W" + str(l)], parameters["b" + str(l)], "sigmoid")
        caches.append(cache)

    AL, cache = linear_activation_forward(A, parameters["W" + str(L)], parameters["b" + str(L)], "relu")
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [0]:
    #Cost Function
    #Squared Error

    # Arguments:
    # AL -- probability vector corresponding to predictions, shape (1, number of examples)
    # Y -- true value

    # Returns:
    # cost -- cross-entropy cost

def compute_cost(AL, Y):  
    m = Y.shape[1]

    cost = (-1/(2 * m)) * np.sum(np.square(Y - AL) )
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

In [0]:
    # The Linear portion of backward propagation for a single layer (layer l)

    # Arguments:
    # dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    # cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    # Returns:
    # dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    # dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    # db -- Gradient of the cost with respect to b (current layer l), same shape as b

def linear_backward(dZ, cache):

    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1/m) * np.dot(dZ, np.transpose(A_prev))
    db = (1/m) * np.sum(dZ,axis = 1,keepdims = True)
    dA_prev = np.dot(np.transpose(W), dZ)
   
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [0]:
    # The backward propagation for the LINEAR->ACTIVATION layer.
    
    # Arguments:
    # dA -- post-activation gradient for current layer l 
    # cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    # activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    # Returns:
    # dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    # dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    # db -- Gradient of the cost with respect to b (current layer l), same shape as b

def linear_activation_backward(dA, cache, activation):

    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
   
    return dA_prev, dW, db

In [0]:
# The backward propagation for the [LINEAR->Sigmoid] * (L-1) -> [LINEAR -> ReLU] group
    
#     Arguments:
#     AL -- Output of the forward propagation (L_model_forward())
#     Y -- True Value
#     caches -- list of caches containing:

    
#     Returns:
#     grads -- A dictionary with the gradients
#              grads["dA" + str(l)] = ... 
#              grads["dW" + str(l)] = ...
#              grads["db" + str(l)] = ... 

def L_model_backward(AL, Y, caches):

    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    dAL = Y - AL # derivative of cost with respect to AL
    
    # Lth layer (ReLU -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]

    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, "relu")
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (Sigmoid -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 

        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+1)], current_cache, "sigmoid")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [0]:
    # Update parameters using gradient descent
    
    # Arguments:
    # parameters -- python dictionary containing your parameters 
    # grads -- python dictionary containing your gradients, output of L_model_backward
    
    # Returns:
    # parameters -- python dictionary containing your updated parameters 
    #               parameters["W" + str(l)] = ... 
    #               parameters["b" + str(l)] = ...

def update_parameters(parameters, grads, learning_rate):

    L = len(parameters) // # number of layers in the neural network

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - np.multiply(learning_rate, grads["dW" + str(l+1)])
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - np.multiply(learning_rate, grads["db" + str(l+1)])

    return parameters